# **Sprint 2 - Desenvolvimento do Modelo** 

Nesta Sprint vamos fazer as seguintes atividades:

* **Escolha do Algoritmo:** Aqui, selecionaremos o algoritmo de classificação mais apropriado para o nosso projeto, com base nas características dos dados.

* **Treinamento do Modelo:** Usando os dados preparados na Sprint 1, treinaremos o modelo de classificação. Isso envolve o ajuste dos parâmetros do algoritmo e a otimização do desempenho.

* **Avaliação de Desempenho:** Avaliaremos o desempenho do modelo com diversas métricas de qualidade, garantindo que ele atenda aos nossos critérios de sucesso.


## Bibliotecas Utilizadas

In [16]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.simplefilter(action='ignore')

## Carregando o Conjunto de Dados

In [2]:
df = pd.read_csv('../datasets/estudo_sono_tratado.csv')
df.head() # Visualizando os primeiros registros

,ID,sexo,idade,ocupacao,horas_de_sono,qualidade_do_sono,nivel_atividade_fisica,nivel_stress,categoria_IMC,pressao_sanguinea,frequencia_cardiaca,passos_diarios,disturbio_do_sono,tem_disturbio_do_sono
0,1,Masculino,27,Engenheiro de Software,6.1,6,42,6,Sobrepeso,126/83,77,4200,Nenhum,False
1,2,Masculino,28,Médico,6.2,6,60,8,Normal,125/80,75,10000,Nenhum,False
2,3,Masculino,28,Médico,6.2,6,60,8,Normal,125/80,75,10000,Nenhum,False
3,4,Masculino,28,Representante de Vendas,5.9,4,30,8,Obeso,140/90,85,3000,Apneia do Sono,True
4,5,Masculino,28,Representante de Vendas,5.9,4,30,8,Obeso,140/90,85,3000,Apneia do Sono,True


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      374 non-null    int64  
 1   sexo                    374 non-null    object 
 2   idade                   374 non-null    int64  
 3   ocupacao                374 non-null    object 
 4   horas_de_sono           374 non-null    float64
 5   qualidade_do_sono       374 non-null    int64  
 6   nivel_atividade_fisica  374 non-null    int64  
 7   nivel_stress            374 non-null    int64  
 8   categoria_IMC           374 non-null    object 
 9   pressao_sanguinea       374 non-null    object 
 10  frequencia_cardiaca     374 non-null    int64  
 11  passos_diarios          374 non-null    int64  
 12  disturbio_do_sono       374 non-null    object 
 13  tem_disturbio_do_sono   374 non-null    bool   
dtypes: bool(1), float64(1), int64(7), object(5

## **Pré-Processamento dos Dados para utilização no Modelo**

In [4]:
# Convertendo a coluna 'pressao_sanguinea' para duas colunas separadas de pressão sistólica e diastólica
df[['pressao_sistolica', 'pressao_diastolica']] = df['pressao_sanguinea'].str.split('/', expand=True)

# Excluindo a coluna original 'pressao_sanguinea'
df.drop('pressao_sanguinea', axis=1, inplace=True)

# Transformando os novos dados em INT:
df['pressao_diastolica'] = df['pressao_diastolica'].astype('int')
df['pressao_sistolica'] = df['pressao_sistolica'].astype('int')
df.head()

,ID,sexo,idade,ocupacao,horas_de_sono,qualidade_do_sono,nivel_atividade_fisica,nivel_stress,categoria_IMC,frequencia_cardiaca,passos_diarios,disturbio_do_sono,tem_disturbio_do_sono,pressao_sistolica,pressao_diastolica
0,1,Masculino,27,Engenheiro de Software,6.1,6,42,6,Sobrepeso,77,4200,Nenhum,False,126,83
1,2,Masculino,28,Médico,6.2,6,60,8,Normal,75,10000,Nenhum,False,125,80
2,3,Masculino,28,Médico,6.2,6,60,8,Normal,75,10000,Nenhum,False,125,80
3,4,Masculino,28,Representante de Vendas,5.9,4,30,8,Obeso,85,3000,Apneia do Sono,True,140,90
4,5,Masculino,28,Representante de Vendas,5.9,4,30,8,Obeso,85,3000,Apneia do Sono,True,140,90


In [5]:
# Criando variaveis dummies
df = pd.get_dummies(df, columns=['sexo', 'ocupacao', 'categoria_IMC', 'disturbio_do_sono'], drop_first=True, dtype='int')
df.head()

,ID,idade,horas_de_sono,qualidade_do_sono,nivel_atividade_fisica,nivel_stress,frequencia_cardiaca,passos_diarios,tem_disturbio_do_sono,pressao_sistolica,...,ocupacao_Gerente,ocupacao_Médico,ocupacao_Professor,ocupacao_Representante de Vendas,ocupacao_Vendedor(a),categoria_IMC_Obeso,categoria_IMC_Peso Normal,categoria_IMC_Sobrepeso,disturbio_do_sono_Insônia,disturbio_do_sono_Nenhum
0,1,27,6.1,6,42,6,77,4200,False,126,...,0,0,0,0,0,0,0,1,0,1
1,2,28,6.2,6,60,8,75,10000,False,125,...,0,1,0,0,0,0,0,0,0,1
2,3,28,6.2,6,60,8,75,10000,False,125,...,0,1,0,0,0,0,0,0,0,1
3,4,28,5.9,4,30,8,85,3000,True,140,...,0,0,0,1,0,1,0,0,0,0
4,5,28,5.9,4,30,8,85,3000,True,140,...,0,0,0,1,0,1,0,0,0,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 27 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ID                                374 non-null    int64  
 1   idade                             374 non-null    int64  
 2   horas_de_sono                     374 non-null    float64
 3   qualidade_do_sono                 374 non-null    int64  
 4   nivel_atividade_fisica            374 non-null    int64  
 5   nivel_stress                      374 non-null    int64  
 6   frequencia_cardiaca               374 non-null    int64  
 7   passos_diarios                    374 non-null    int64  
 8   tem_disturbio_do_sono             374 non-null    bool   
 9   pressao_sistolica                 374 non-null    int32  
 10  pressao_diastolica                374 non-null    int32  
 11  sexo_Masculino                    374 non-null    int32  
 12  ocupacao

In [7]:
X = df.drop(['tem_disturbio_do_sono'], axis=1)
y = df['tem_disturbio_do_sono']

## **Escolha do Algoritmo**

Neste projeto, serão utilizados três algoritmos de classificação para modelar e prever os resultados desejados. Cada um desses algoritmos tem características distintas, oferecendo diferentes vantagens em termos de desempenho e interpretabilidade.

### 1. RandomForest Classifier

O RandomForest Classifier é um modelo baseado em conjunto que combina as previsões de várias árvores de decisão para obter uma previsão mais robusta e geral. 

Este modelo é conhecido por sua eficácia em lidar com uma variedade de conjuntos de dados e sua resistência ao overfitting.

### 2. Logistic Regression Classifier

O Logistic Regression Classifier é uma técnica de regressão utilizada para problemas de classificação binária. Apesar da sua simplicidade, a Regressão Logística é rápida e fácil de interpretar. 

É especialmente eficaz quando a relação entre as features e a variável de destino é predominantemente linear.

### 3. Decision Tree Classifier

O Decision Tree Classifier é um modelo de árvore de decisão que divide iterativamente o conjunto de dados em subconjuntos mais puros. 

Apesar de sua simplicidade e interpretabilidade, as árvores de decisão podem ser propensas a sobreajuste, ajustando-se demais aos dados de treinamento.

In [18]:
# Classificador RandomForest
rf_classifier = RandomForestClassifier(random_state=42)

# Avaliação usando validação cruzada
rf_scores = cross_val_score(rf_classifier, X, y, cv=10, scoring='accuracy')

print("RandomForest Classifier:")
print("Cross-Validation Scores:", rf_scores)
print("Mean Accuracy:", rf_scores.mean())

# Classificador Logistic Regression
lr_classifier = LogisticRegression(random_state=42)

# Avaliação usando validação cruzada
lr_scores = cross_val_score(lr_classifier, X, y, cv=5, scoring='accuracy')

print("\nLogistic Regression Classifier:")
print("Cross-Validation Scores:", lr_scores)
print("Mean Accuracy:", lr_scores.mean())

# Classificador Decision Tree
dt_classifier = DecisionTreeClassifier(random_state=42)

# Avaliação usando validação cruzada
dt_scores = cross_val_score(dt_classifier, X, y, cv=5, scoring='accuracy')

print("\nDecision Tree Classifier:")
print("Cross-Validation Scores:", dt_scores)
print("Mean Accuracy:", dt_scores.mean())

RandomForest Classifier:
Cross-Validation Scores: [1.         0.97368421 1.         1.         1.         1.
 1.         1.         0.89189189 1.        ]
Mean Accuracy: 0.9865576102418208

Logistic Regression Classifier:
Cross-Validation Scores: [0.78666667 0.93333333 0.98666667 0.78666667 0.90540541]
Mean Accuracy: 0.8797477477477477

Decision Tree Classifier:
Cross-Validation Scores: [1. 1. 1. 1. 1.]
Mean Accuracy: 1.0


## **Treinamento do Modelo**

## **Avaliação de Desempenho**

## **Conclusão**